In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from functions import merge_data

# try loading the merged data
ahrf_data = 'data/hrsa/data_AHRF_2018-2019/processed/df_renamed.pkl'
usafacts_data_cases = 'data/usafacts/confirmed_cases_mar22.csv'
usafacts_data_deaths = 'data/usafacts/deaths_mar22.csv'
diabetes = 'data/diabetes/DiabetesAtlasCountyData.csv'
voting = 'data/voting/county_voting_processed.pkl'
df = merge_data.merge_data(ahrf_data=ahrf_data, 
                           usafacts_data_cases=usafacts_data_cases,
                           usafacts_data_deaths=usafacts_data_deaths,
                           medicare_group="All Beneficiaries",
                           voting=voting,
                           diabetes=diabetes) # also cleans usafacts data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
